In [6]:
from konlpy.tag import Okt
from glob import glob
import ujson as json

twitter = Okt()

In [9]:
keywords = ["대선", "문재인", "안철수", "홍준표", "유승민", "심상정"]
a = 154092
for keyword in keywords:
    files = glob("crawlwed_articles/{}/*.json".format(keyword))
    totalNum = 0
    for file in files:
        with open(file) as f:
            totalNum += len(json.load(f))
    print("{}: 총 {}개의 기사 (전체의 {:.4f}%)".format(keyword, totalNum, totalNum/a*100))

대선: 총 208076개의 기사 (전체의 135.0336%)
문재인: 총 208076개의 기사 (전체의 135.0336%)
안철수: 총 208076개의 기사 (전체의 135.0336%)
홍준표: 총 208076개의 기사 (전체의 135.0336%)
유승민: 총 208076개의 기사 (전체의 135.0336%)
심상정: 총 208076개의 기사 (전체의 135.0336%)


In [11]:
with open(files[0]) as f:
    data = json.load(f)

In [34]:
articleList = []
titles = set()
articleNum = 0

for keyword in keywords:
    files = glob("cralwed_articles/{}/*.json".format(keyword))
    for file in files:
        with open(file) as f:
            articles = json.load(f)
        for article in articles:
            if article['title'] not in titles:
                # 기사 중복되지 않는 것들만 처리
                titles.add(article['title'])
                articleList.append(article)
                articleNum += 1
                
                if articleNum % 10000 == 0:
                    print(articleNum)

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000


In [21]:
from collections import defaultdict
related_articles = defaultdict(list)

for article in articleList:
    for keyword in keywords[1:]:
        if keyword in article['title']:
            related_articles[keyword].append(article['title'])

with open("related_articles.pkl", "wb") as f:
    pickle.dump(related_articles, f)

In [36]:
import pickle

with open("articles.pkl", "wb") as f:
    pickle.dump(articleList, f)

In [2]:
import pickle

with open("articles.pkl", "rb") as f:
    articleList = pickle.load(f)

In [3]:
spamTag = set(['URL', 'Email', 'KoreanParticle', 'Foreign', 'Number', 'Punctuation'])
wordsList = []
for i, article in enumerate(articleList):
    if i % 10000 == 0:
        print(i)

    words = []
    p = twitter.pos(article['text'], stem=True, norm=True)
    for pos in p:
        if pos[1] not in spamTag:
            words.append(pos[0])
    wordsList.append(words)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000


In [4]:
from gensim.models.word2vec import Word2Vec

model = Word2Vec(wordsList, size=200, min_count=5, workers=5, sg=1)

In [5]:
model.wv.save_word2vec_format('word2vec.txt')

In [7]:
a = model['문재인']
b = model['대통령']

In [16]:
candidates = ['문재인', '홍준표', '안철수', '유승민', '심상정']
tendencies = ['진보', '보수']
for tendency in tendencies:
    for candidate in candidates:
        print("Similarity between {} and {}: {}".format(candidate, tendency, model.wv.similarity(candidate, tendency)))
    print()

Similarity between 문재인 and 진보: 0.34033064805568114
Similarity between 홍준표 and 진보: 0.24932029585687634
Similarity between 안철수 and 진보: 0.2935491661969409
Similarity between 유승민 and 진보: 0.3013114592999173
Similarity between 심상정 and 진보: 0.36947003451293675

Similarity between 문재인 and 보수: 0.33152449313652027
Similarity between 홍준표 and 보수: 0.4095070890920629
Similarity between 안철수 and 보수: 0.32456712444940394
Similarity between 유승민 and 보수: 0.3791166938994421
Similarity between 심상정 and 보수: 0.25648218122290956

